# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [248]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. 
'''

## 2. Generate the mathematical model

In [276]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [277]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Drug Production

**Sets and Indices:**

*  $i \in \{1,2\}$: Index for raw materials.
*  $j \in \{1,2\}$: Index for drugs.

**Parameters:**

*  $c_i$: Purchasing cost per unit of raw material $i$.
*  $o_j$: Operational cost per unit of drug $j$.
*  $s_j$: Selling price per unit of drug $j$.
*  $B$: Total budget for purchasing raw materials and operating the production process.
*  $K_j$: Production capacity for drug $j$ (in units).
*  $S_i$: Storage capacity for raw material $i$ (in units).
*  $R_{ij}$: Amount of raw material $i$ required to produce one unit of drug $j$.
*  $A_j$: Required amount of active ingredient in one unit of drug $j$.
*  $E_i$: Nominal amount of active ingredient extracted from one unit of raw material $i$.
*  $\alpha_1 = 0.005$: Variability of active ingredient extraction from raw material 1.
*  $\alpha_2 = 0.02$: Variability of active ingredient extraction from raw material 2.

**Decision Variables:**

*  $x_i$: Amount of raw material $i$ to purchase (in units).
*  $y_j$: Amount of drug $j$ to produce (in units).

**Objective Function:**

Maximize the total profit:

$$ \max \sum_{j=1}^{2} s_j y_j - \sum_{i=1}^{2} c_i x_i - \sum_{j=1}^{2} o_j y_j $$

**Constraints:**

1. **Budget Constraint:** The total cost of purchasing raw materials and operating the production process cannot exceed the budget. 
 $$  \sum_{i=1}^{2} c_i x_i + \sum_{j=1}^{2} o_j y_j \leq B $$

2. **Capacity Constraints:** The production of each drug cannot exceed its capacity.
$$  y_j \leq K_j, \quad \forall j \in \{1,2\}  $$

3. **Storage Constraints:** The amount of each raw material purchased cannot exceed its storage capacity.
$$ x_i \leq S_i, \quad \forall i \in \{1,2\}  $$

4. **Raw Material Requirement Constraints:** The amount of raw material purchased must be sufficient to produce the drugs.
$$  x_i \geq \sum_{j=1}^{2} R_{ij} y_j, \quad \forall i \in \{1,2\}  $$

5. **Active Ingredient Requirement Constraints:** The amount of active ingredient extracted from the purchased raw materials must satisfy the required amount for the drug production, considering the variability in extraction.
$$  (1 - \alpha_i)E_i x_i  \geq  \sum_{j=1}^{2} A_j y_j, \quad \forall i \in \{1,2\}  $$

**Non-negativity Constraints:**
$$ x_i \geq 0, \quad \forall i \in \{1,2\} $$
$$ y_j \geq 0, \quad \forall j \in \{1,2\} $$

This model aims to find the optimal amount of each raw material to purchase and each drug to produce to maximize profit while satisfying all budget, capacity, storage, raw material, and active ingredient requirements. 


## 3. Generate the pyomo code

In [278]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [279]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data
c = {1: 10, 2: 15}  # Cost of raw materials
o = {1: 5, 2: 8}  # Operational cost of drugs
s = {1: 30, 2: 45}  # Selling price of drugs
B = 1000  # Total budget
K = {1: 100, 2: 150}  # Production capacity for drugs
S = {1: 200, 2: 250}  # Storage capacity for raw materials
R = {(1, 1): 0.5, (1, 2): 0.8, (2, 1): 0.3, (2, 2): 0.6}  # Raw material requirement
A = {1: 0.2, 2: 0.3}  # Active ingredient requirement
E = {1: 0.4, 2: 0.7}  # Nominal amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Variability of active ingredient extraction

# Create a model
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material to purchase
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)  # Amount of drug to produce

# Objective function
model.profit = pyo.Objective(expr=sum(s[j] * model.y[j] for j in model.j) - sum(c[i] * model.x[i] for i in model.i) - sum(o[j] * model.y[j] for j in model.j), sense=pyo.maximize)

# Constraints
model.budget_constraint = pyo.Constraint(expr=sum(c[i] * model.x[i] for i in model.i) + sum(o[j] * model.y[j] for j in model.j) <= B)

model.capacity_constraints = pyo.ConstraintList()
for j in model.j:
    model.capacity_constraints.add(expr=model.y[j] <= K[j])

model.storage_constraints = pyo.ConstraintList()
for i in model.i:
    model.storage_constraints.add(expr=model.x[i] <= S[i])

model.raw_material_constraints = pyo.ConstraintList()
for i in model.i:
    model.raw_material_constraints.add(expr=model.x[i] >= sum(R[i, j] * model.y[j] for j in model.j))

model.active_ingredient_constraints = pyo.ConstraintList()
for i in model.i:
    model.active_ingredient_constraints.add(expr=(1 - alpha[i]) * E[i] * model.x[i] >= sum(A[j] * model.y[j] for j in model.j))

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"Profit = {pyo.value(model.profit):.2f}")
print("Raw material purchases:")
for i in model.i:
    print(f"x[{i}] = {pyo.value(model.x[i]):.2f}")
print("Drug production:")
for j in model.j:
    print(f"y[{j}] = {pyo.value(model.y[j]):.2f}")
```

**Sample data used:**

*   Costs of raw materials, operational costs, selling prices, budget, production capacities, storage capacities, raw material requirements, active ingredient requirements, nominal amounts of active ingredient extracted, and variabilities of active ingredient extraction are all defined as sample data in the code. 

This Pyomo code defines the sets, parameters, variables, objective function, and constraints of the mathematical optimization model for drug production. It uses the GLPK solver to find the optimal solution that maximizes profit while satisfying all constraints. The results, including the optimal profit, raw material purchases, and drug production amounts, are then printed. 


## 4. Input problem data and try running the generated code

In [280]:
import pyomo.environ as pyo

# Sample data
c = {1: 10, 2: 15}  # Cost of raw materials
o = {1: 5, 2: 8}  # Operational cost of drugs
s = {1: 30, 2: 45}  # Selling price of drugs
B = 1000  # Total budget
K = {1: 100, 2: 150}  # Production capacity for drugs
S = {1: 200, 2: 250}  # Storage capacity for raw materials
R = {(1, 1): 0.5, (1, 2): 0.8, (2, 1): 0.3, (2, 2): 0.6}  # Raw material requirement
A = {1: 0.2, 2: 0.3}  # Active ingredient requirement
E = {1: 0.4, 2: 0.7}  # Nominal amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Variability of active ingredient extraction

# Create a model
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material to purchase
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)  # Amount of drug to produce

# Objective function
model.profit = pyo.Objective(expr=sum(s[j] * model.y[j] for j in model.j) - sum(c[i] * model.x[i] for i in model.i) - sum(o[j] * model.y[j] for j in model.j), sense=pyo.maximize)

# Constraints
model.budget_constraint = pyo.Constraint(expr=sum(c[i] * model.x[i] for i in model.i) + sum(o[j] * model.y[j] for j in model.j) <= B)

model.capacity_constraints = pyo.ConstraintList()
for j in model.j:
    model.capacity_constraints.add(expr=model.y[j] <= K[j])

model.storage_constraints = pyo.ConstraintList()
for i in model.i:
    model.storage_constraints.add(expr=model.x[i] <= S[i])

model.raw_material_constraints = pyo.ConstraintList()
for i in model.i:
    model.raw_material_constraints.add(expr=model.x[i] >= sum(R[i, j] * model.y[j] for j in model.j))

model.active_ingredient_constraints = pyo.ConstraintList()
for i in model.i:
    model.active_ingredient_constraints.add(expr=(1 - alpha[i]) * E[i] * model.x[i] >= sum(A[j] * model.y[j] for j in model.j))

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"Profit = {pyo.value(model.profit):.2f}")
print("Raw material purchases:")
for i in model.i:
    print(f"x[{i}] = {pyo.value(model.x[i]):.2f}")
print("Drug production:")
for j in model.j:
    print(f"y[{j}] = {pyo.value(model.y[j]):.2f}")

Optimal solution:
Profit = 1065.39
Raw material purchases:
x[1] = 34.60
x[2] = 20.65
Drug production:
y[1] = 68.85
y[2] = 0.00


## 5. Correct the code to verify model viability (optional)

In [265]:
#This is correct code to compare, but with some parameters input manually.
from pyomo.environ import *

# Sample data (replace with actual data)
c = {1: 100, 2: 199.90}  # Cost per unit of raw material
p = {1: 6200, 2: 6900}  # Selling price per unit of drug
o = {1: 700, 2: 800}   # Operational cost per unit of drug
a = {
    (1, 1): 0.01,
    (1, 2): 0.01,
    (2, 1): 0.02,
    (2, 2): 0.02
}  # Nominal amount of active ingredient
alpha = {1: 0.005, 2: 0.02}  # Variability percentage
r = {1: 0.5, 2: 0.6}  # Required active ingredient per unit
B = 100000  # Budget
C = 1000  # Storage capacity #ALTERED
M = 2000  # Available manpower hours
E = 800  # Available equipment hours
m = {1: 90, 2: 100}  # Manpower hours per unit
e = {1: 40, 2: 50}  # Equipment hours per unit

# Model creation
model = ConcreteModel()

# Sets
model.I = Set(initialize=c.keys())  # Raw materials
model.J = Set(initialize=p.keys())  # Drugs

# Parameters
model.c = Param(model.I, initialize=c)
model.p = Param(model.J, initialize=p)
model.o = Param(model.J, initialize=o)
model.a = Param(model.I, model.J, initialize=a)
model.alpha = Param(model.I, initialize=alpha)
model.r = Param(model.J, initialize=r)
model.B = Param(initialize=B)
#model.C = Param(model.I, initialize=C)
model.M = Param(initialize=M)
model.E = Param(initialize=E)
model.m = Param(model.J, initialize=m)
model.e = Param(model.J, initialize=e)

# Decision variables
model.x = Var(model.I, domain=NonNegativeReals)
model.y = Var(model.J, domain=NonNegativeReals)

# Objective function
model.profit = Objective(
    expr=sum((model.p[j] - model.o[j]) * model.y[j] for j in model.J) -
    sum(model.c[i] * model.x[i] for i in model.I),
    sense=maximize
)

# Constraints
model.budget_constraint = Constraint(
    expr=sum(model.c[i] * model.x[i] for i in model.I) +
    sum(model.o[j] * model.y[j] for j in model.J) <= model.B
)
model.storage_constraint = Constraint(expr=sum(model.x[i] for i in model.I)<=C)
model.manpower_constraint = Constraint(expr=sum(model.m[j] * model.y[j] for j in model.J) <= model.M)
model.equipment_constraint = Constraint(expr=sum(model.e[j] * model.y[j] for j in model.J) <= model.E)
model.active_ingredient_constraint = Constraint(expr=(0.01*0.995*model.x[1] + 0.02*0.98*model.x[2] - 0.5*model.y[1] - 0.6* model.y[2]>=0))

# Solve the model
solver = SolverFactory('glpk')  # You can change the solver as needed
results = solver.solve(model)

# Print results (you might want to improve formatting)
print(results)

# Accessing results
print("Optimal profit:", model.profit())
for i in model.I:
    print(f"Quantity of raw material {i} to purchase: {model.x[i]():.2f}")
for j in model.J:
    print(f"Quantity of drug {j} to produce: {model.y[j]():.2f}")


Problem: 
- Name: unknown
  Lower bound: 8294.56683928728
  Upper bound: 8294.56683928728
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 14
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03960728645324707
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal profit: 8294.566839287349
Quantity of raw material 1 to purchase: 877.73
Quantity of raw material 2 to purchase: 0.00
Quantity of drug 1 to produce: 17.47
Quantity of drug 2 to produce: 0.00


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [281]:
print(response.text)

## Mathematical Optimization Model for Drug Production

**Sets and Indices:**

*  $i \in \{1,2\}$: Index for raw materials.
*  $j \in \{1,2\}$: Index for drugs.

**Parameters:**

*  $c_i$: Purchasing cost per unit of raw material $i$.
*  $o_j$: Operational cost per unit of drug $j$.
*  $s_j$: Selling price per unit of drug $j$.
*  $B$: Total budget for purchasing raw materials and operating the production process.
*  $K_j$: Production capacity for drug $j$ (in units).
*  $S_i$: Storage capacity for raw material $i$ (in units).
*  $R_{ij}$: Amount of raw material $i$ required to produce one unit of drug $j$.
*  $A_j$: Required amount of active ingredient in one unit of drug $j$.
*  $E_i$: Nominal amount of active ingredient extracted from one unit of raw material $i$.
*  $\alpha_1 = 0.005$: Variability of active ingredient extraction from raw material 1.
*  $\alpha_2 = 0.02$: Variability of active ingredient extraction from raw material 2.

**Decision Variables:**

*  $x_i$: Amount of r

In [282]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data
c = {1: 10, 2: 15}  # Cost of raw materials
o = {1: 5, 2: 8}  # Operational cost of drugs
s = {1: 30, 2: 45}  # Selling price of drugs
B = 1000  # Total budget
K = {1: 100, 2: 150}  # Production capacity for drugs
S = {1: 200, 2: 250}  # Storage capacity for raw materials
R = {(1, 1): 0.5, (1, 2): 0.8, (2, 1): 0.3, (2, 2): 0.6}  # Raw material requirement
A = {1: 0.2, 2: 0.3}  # Active ingredient requirement
E = {1: 0.4, 2: 0.7}  # Nominal amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Variability of active ingredient extraction

# Create a model
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material to purchase
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)  # Amount of drug to produce

# Objective function
model.profit = pyo.Obj